# Experiment 03: Going deeper

In this experiment we will study how different band configurations affect the training process and end results.

### Experiment variations:

- E03-4: The 4 MS WorldView-2 bands that are also available in GeoEye-1 (MS1 array RGB+NIR1)
 - In this variation we will also validate performance on the GeoEye-1 validation set
- E03-3: Only the 3 RGB bands from WorldView-2
 - Also validated on the GeoEye-1 validation set
 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *
from modules.evaluation import *

from modules.logging import *
from modules.train import *

import time

# Check GPUs and enable dynamic GPU memory use:",
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            # Prevent TensorFlow from allocating all memory of all GPUs:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [2]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e03-8', 'e03-6', 'e03-4', 'e03-3']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[3]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRE_BUILD = True          # Step 1 of the training process
PRETRAIN = False          # Step 1 of the training process
GAN_BUILD = True          # Step 1 of the training process
GAN_TRAIN = False         # Step 2 of the training process
PRE_EVALUATE_LAST = False
GAN_EVALUATE_LAST = True
PRE_EVALUATE_HISTORY = False
GAN_EVALUATE_HISTORY = False

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e03'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 2.0, 
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.127841.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
    NEW_PIXEL_SIZE_PAN = 0.5
    RESIZE_DIR = DATA_PATH + '-resized'
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e03-8': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e03-6': {'train': 'WV02', 'val': ['WV02'], 'test': ['WV02']}, 
               'e03-4': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}, 
               'e03-3': {'train': 'WV02', 'val': ['WV02', 'GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e03-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e03-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e03-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e03-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e03-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                        'e03-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                  if k not in ['Coastal', 'NIR2']}, 
                        'e03-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                  if k in ['Blue', 'Green', 'Red', 'NIR']},
                        'e03-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                  if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e03-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e03-8': {None: None},                                   # not enough bands in GE01
                        'e03-6': {None: None},                                    # not enough bands in GE01
                        'e03-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                        'e03-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                  if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e03-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 23 # Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 0     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 1
VAL_N_BATCHES_SAVE = 1

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 0
# Proportion of val batches that will go through ma and niqe metric calculation
# MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
MA_NIQE_PROPORTION = 1  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_DIRS = {'e03-3': LOGS_EXP_DIR + '/models/' + 'e03-3-pre_20210306-143849/'
                        }
PRETRAIN_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIRS[EXPERIMENT]
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_DIR + EXPERIMENT + '-pre-400.h5'

# Path to the gan-training weights that will be 
GAN_WEIGHTS_DIRS = {'e03-3': LOGS_EXP_DIR + '/models/' + 'e03-3-gan_20210310-092747/'
                   }
GAN_WEIGHTS_DIR = GAN_WEIGHTS_DIRS[EXPERIMENT]
GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-399.h5'
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = False
METRIC_NIQE = False
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
#################################################################################################################

### EVALUTAION ##################################################################################################
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    METRIC_MA = True
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        

EVAL_STEPS_PER_EPOCH = 'all'
EVAL_N_EPOCHS = 400
#EVAL_SENSOR = 'WV02'
EVAL_PER_IMAGE = True
    
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        
#if PRE_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 1
#    EVAL_PREFIX = EXPERIMENT + '-pre-'
#elif GAN_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = GAN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 0
#    EVAL_PREFIX = EXPERIMENT + '-gan-'
    
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 22, 'val': 19, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: True
NIQE: True
Perceptual Index (PI): True


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [3]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 129280, 'val': 8066, 'test': 9222}
Total number of tiles: 146568


### 2.3 Tile generation to disk

In [4]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [5]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [6]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 339.8 and sd 128.3 from json file @ data/toulon-laspezia-tiles/e03/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [7]:
SHUFFLE_BUFFER_SIZE = {'train': n_tiles['train'],  # 100
                       'val': n_tiles['val'],  # 100
                       'test': n_tiles['test']}  # 100

train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.1996114807902925
Output (float) min, max: -0.19902573299693777 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [8]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [9]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [10]:
if PRE_BUILD:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [11]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [12]:
if GAN_BUILD:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH)

Starting matlab.engine ...
matlab.engine started


## 7. GAN training

In [13]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

## 8. Evaluation

### 8.1 Data input pipelines for final evaluation

The pipeline is modified to include the file paths of the tiles/patches so that it is possible to log performance metrics for individual files and by extension for individual satellite images.

#### 8.1.1 Validation set

In [14]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'val'
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=False,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_filepath_cache'), 
                                    repeat=False, 
                                    shuffle=False, 
                                    shuffle_buffer_size=0, #SHUFFLE_BUFFER_SIZE[train_val_test], 
                                    include_file_paths=True)
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


#### 8.1.2 Test set

In [15]:
train_val_test = 'test'
ds_test = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_test[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                     batch_size=BATCH_SIZE[train_val_test], 
                                     ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                     pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                     sensor=sensor,
                                     band_selection=band_indices, 
                                     mean_correction=train_tiles_mean,
                                     cache_memory=False,
                                     cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_filepath_cache'), 
                                     repeat=False, 
                                     shuffle=False, 
                                     shuffle_buffer_size=0)
    ds_test[sensor] = ds_test[sensor].get_dataset()
print(ds_test.keys())

dict_keys(['WV02', 'GE01'])


### 8.2 Evaluate last epoch

In [16]:
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    val_or_test = 'val'
    # SENSORS = {'val': ['GE01']}

    # Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
    #if METRIC_MA:
    #    PRE_GAN = ['gan']
    #else:
    #    PRE_GAN = ['pre', 'gan']
    
    if PRE_EVALUATE_LAST and not GAN_EVALUATE_LAST:
        PRE_GAN = ['pre']
    if not PRE_EVALUATE_LAST and GAN_EVALUATE_LAST:
        PRE_GAN = ['gan']

    print(PRE_GAN, SENSORS[val_or_test])
    for pre_gan in PRE_GAN:
        
        for sensor in SENSORS[val_or_test]:
            # if sensor == 'GE01':
            #     continue
                
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
            else:
                gan_model.G.load_weights(GAN_WEIGHTS_PATH)

            print(pre_gan, sensor)
            start = time.time()
            results_df = esrgan_evaluate(model=gan_model, 
                                         dataset=ds_val[sensor], 
                                         steps='all', 
                                         per_image=True, 
                                         write_csv=True,
                                         csv_path=str(LOGS_EXP_DIR + '/csv/' + 'final_epoch-' 
                                                      + pre_gan + '-' + val_or_test + '-' + sensor + '.csv'), 
                                         verbose=1
                                        )
            end = time.time()
            print(str((end - start) / 60), 'minutes')

['gan'] ['WV02', 'GE01']
gan WV02
Computed 8 images in  230.6237277984619 seconds
Last image: {'G_pixel_loss': 0.00011829865979962051, 'G_perceptual_loss': 1.928391695022583, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.49898910522461, 'SSIM': 0.8887779116630554, 'Ma': 4.78265380859375, 'NIQE': 4.000634670257568, 'PI': 4.608990669250488}
Computed 8 images in  123.49849796295166 seconds
Last image: {'G_pixel_loss': 0.00017882234533317387, 'G_perceptual_loss': 3.0608325004577637, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.88837432861328, 'SSIM': 0.8114356398582458, 'Ma': 4.705907344818115, 'NIQE': 4.313650131225586, 'PI': 4.803871154785156}
Computed 8 images in  122.88542652130127 seconds
Last image: {'G_pixel_loss': 0.0001599666429683566, 'G_perceptual_loss': 1.1947367191314697, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 34.34992218017578, 'SSIM': 0.8595301508903503, 'Ma': 4.6977148056030

Computed 8 images in  122.7444396018982 seconds
Last image: {'G_pixel_loss': 0.00018467145855538547, 'G_perceptual_loss': 1.7060421705245972, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.490840911865234, 'SSIM': 0.7693684697151184, 'Ma': 4.3834404945373535, 'NIQE': 3.463526725769043, 'PI': 4.540042877197266}
Computed 8 images in  122.91191244125366 seconds
Last image: {'G_pixel_loss': 0.0001476113684475422, 'G_perceptual_loss': 2.4586052894592285, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 31.71380043029785, 'SSIM': 0.8542399406433105, 'Ma': 4.707452774047852, 'NIQE': 4.012159824371338, 'PI': 4.652353286743164}
Computed 8 images in  122.9254469871521 seconds
Last image: {'G_pixel_loss': 1.8180508050136268e-05, 'G_perceptual_loss': 0.24984923005104065, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 49.474491119384766, 'SSIM': 0.9965602159500122, 'Ma': 3.0672128200531006, 'NIQE': 9.70849800109863

Computed 8 images in  120.00787663459778 seconds
Last image: {'G_pixel_loss': 9.605415107216686e-05, 'G_perceptual_loss': 1.6744749546051025, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.69342803955078, 'SSIM': 0.9237133860588074, 'Ma': 4.389993190765381, 'NIQE': 4.758673667907715, 'PI': 5.184340476989746}
Computed 8 images in  119.53715014457703 seconds
Last image: {'G_pixel_loss': 0.00010811091487994418, 'G_perceptual_loss': 1.780583381652832, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 35.43628692626953, 'SSIM': 0.8846434354782104, 'Ma': 4.746574878692627, 'NIQE': 3.9266629219055176, 'PI': 4.590044021606445}
Computed 8 images in  119.92058634757996 seconds
Last image: {'G_pixel_loss': 4.8211066314252093e-05, 'G_perceptual_loss': 0.8606562614440918, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 37.40133285522461, 'SSIM': 0.9626847505569458, 'Ma': 4.893869400024414, 'NIQE': 6.30246114730835, '

Computed 8 images in  121.12221765518188 seconds
Last image: {'G_pixel_loss': 0.00010493744048289955, 'G_perceptual_loss': 1.2105915546417236, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 38.27851486206055, 'SSIM': 0.8729701638221741, 'Ma': 4.382415294647217, 'NIQE': 3.770354747772217, 'PI': 4.6939697265625}
Computed 8 images in  121.23545289039612 seconds
Last image: {'G_pixel_loss': 0.0002680338511709124, 'G_perceptual_loss': 2.9430105686187744, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.938444137573242, 'SSIM': 0.6469516158103943, 'Ma': 4.780935764312744, 'NIQE': 3.7152717113494873, 'PI': 4.467167854309082}
Computed 8 images in  121.22256541252136 seconds
Last image: {'G_pixel_loss': 0.00029542026459239423, 'G_perceptual_loss': 2.6836211681365967, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.62110710144043, 'SSIM': 0.6861420273780823, 'Ma': 4.689966201782227, 'NIQE': 3.55167818069458, '

Computed 8 images in  119.77026748657227 seconds
Last image: {'G_pixel_loss': 3.164916779496707e-05, 'G_perceptual_loss': 0.8612678647041321, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 48.759727478027344, 'SSIM': 0.990436315536499, 'Ma': 2.9346327781677246, 'NIQE': 8.907946586608887, 'PI': 7.98665714263916}
Computed 8 images in  120.27693462371826 seconds
Last image: {'G_pixel_loss': 0.00021515022672247142, 'G_perceptual_loss': 3.5011401176452637, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 30.1811580657959, 'SSIM': 0.6986181139945984, 'Ma': 4.728570461273193, 'NIQE': 3.342444896697998, 'PI': 4.306937217712402}
Computed 8 images in  119.61651158332825 seconds
Last image: {'G_pixel_loss': 0.00016331716324202716, 'G_perceptual_loss': 2.8032381534576416, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.45567321777344, 'SSIM': 0.7390490174293518, 'Ma': 4.555286407470703, 'NIQE': 4.518917560577393, '

...

Computed 8 images in  119.50293493270874 seconds
Last image: {'G_pixel_loss': 0.00028051447588950396, 'G_perceptual_loss': 5.271384239196777, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 28.860431671142578, 'SSIM': 0.7500616908073425, 'Ma': 4.315908908843994, 'NIQE': 5.216742515563965, 'PI': 5.450416564941406}
Computed 8 images in  119.86334228515625 seconds
Last image: {'G_pixel_loss': 0.00015716570487711579, 'G_perceptual_loss': 2.140906810760498, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 34.673072814941406, 'SSIM': 0.8900483250617981, 'Ma': 4.631618499755859, 'NIQE': 6.356110572814941, 'PI': 5.862246036529541}
Computed 8 images in  119.86818623542786 seconds
Last image: {'G_pixel_loss': 0.0002970355562865734, 'G_perceptual_loss': 4.255600452423096, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.036884307861328, 'SSIM': 0.6076534390449524, 'Ma': 4.447304725646973, 'NIQE': 5.193431854248047, 

Computed 8 images in  119.35835766792297 seconds
Last image: {'G_pixel_loss': 0.00034598633646965027, 'G_perceptual_loss': 4.166926383972168, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.490129470825195, 'SSIM': 0.6921820640563965, 'Ma': 4.838318347930908, 'NIQE': 3.45198917388916, 'PI': 4.306835174560547}
Computed 8 images in  119.40474700927734 seconds
Last image: {'G_pixel_loss': 0.00017197652778122574, 'G_perceptual_loss': 1.8211398124694824, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 33.585693359375, 'SSIM': 0.9142618775367737, 'Ma': 4.9069952964782715, 'NIQE': 3.9485435485839844, 'PI': 4.520773887634277}
Computed 8 images in  119.41759085655212 seconds
Last image: {'G_pixel_loss': 0.00025704241124913096, 'G_perceptual_loss': 3.0239310264587402, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.995704650878906, 'SSIM': 0.6997461915016174, 'Ma': 4.895003795623779, 'NIQE': 3.303727865219116,

Computed 8 images in  120.26455044746399 seconds
Last image: {'G_pixel_loss': 0.0003417734114918858, 'G_perceptual_loss': 4.014701843261719, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 27.540536880493164, 'SSIM': 0.5304626822471619, 'Ma': 4.751424789428711, 'NIQE': 3.193502902984619, 'PI': 4.221038818359375}
Computed 8 images in  120.10058355331421 seconds
Last image: {'G_pixel_loss': 0.0005184693727642298, 'G_perceptual_loss': 4.873740196228027, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 24.787683486938477, 'SSIM': 0.32659488916397095, 'Ma': 4.744697093963623, 'NIQE': 3.648970603942871, 'PI': 4.452136993408203}
Computed 8 images in  119.6752655506134 seconds
Last image: {'G_pixel_loss': 0.0003742247063200921, 'G_perceptual_loss': 4.600465774536133, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 26.962892532348633, 'SSIM': 0.5885156989097595, 'Ma': 4.531907081604004, 'NIQE': 3.1527700424194336, '

Computed 8 images in  119.27873158454895 seconds
Last image: {'G_pixel_loss': 0.0006279440713115036, 'G_perceptual_loss': 4.011011600494385, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 22.74141502380371, 'SSIM': 0.5449198484420776, 'Ma': 4.3346428871154785, 'NIQE': 3.6787567138671875, 'PI': 4.672057151794434}
Computed 8 images in  119.24418091773987 seconds
Last image: {'G_pixel_loss': 0.0005768529954366386, 'G_perceptual_loss': 2.9368085861206055, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 23.390727996826172, 'SSIM': 0.7355751395225525, 'Ma': 4.542781352996826, 'NIQE': 3.8216285705566406, 'PI': 4.639423370361328}
Computed 8 images in  119.28470635414124 seconds
Last image: {'G_pixel_loss': 0.00038183576543815434, 'G_perceptual_loss': 0.4451347887516022, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 29.890419006347656, 'SSIM': 0.8907016515731812, 'Ma': 2.9589695930480957, 'NIQE': 9.4295206069946

Computed 8 images in  118.85733842849731 seconds
Last image: {'G_pixel_loss': 0.0006335204816423357, 'G_perceptual_loss': 4.011045455932617, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 22.57668113708496, 'SSIM': 0.7632168531417847, 'Ma': 4.669271945953369, 'NIQE': 4.898540019989014, 'PI': 5.114634037017822}
Computed 8 images in  118.84110641479492 seconds
Last image: {'G_pixel_loss': 0.0011039279634132981, 'G_perceptual_loss': 8.288420677185059, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 17.98174476623535, 'SSIM': 0.4126671850681305, 'Ma': 4.575122356414795, 'NIQE': 3.799151659011841, 'PI': 4.6120147705078125}
Computed 8 images in  119.15886211395264 seconds
Last image: {'G_pixel_loss': 0.0015373823698610067, 'G_perceptual_loss': 5.240137100219727, 'G_generator_loss': nan, 'G_loss_total': nan, 'D_loss_total': 0.0, 'PSNR': 15.855964660644531, 'SSIM': 0.3895353078842163, 'Ma': 4.588397026062012, 'NIQE': 5.154095649719238, 'PI

### 8.3 Evaluate every kth epoch

In [17]:
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    val_or_test = 'val'
    
    # Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
    if METRIC_MA:
        PRE_GAN = ['gan']
    else:
        PRE_GAN = ['pre', 'gan']

    for pre_gan in PRE_GAN:
        for sensor in SENSORS[val_or_test]:
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                model_weights_dir = PRETRAIN_WEIGHTS_DIR
                eval_first_step = 1
                eval_prefix = EXPERIMENT + '-pre-'
            else:
                model_weights_dir = GAN_WEIGHTS_DIR
                eval_first_step = 0
                eval_prefix = EXPERIMENT + '-gan-'

            esrgan_epoch_evaluator(gan_model,
                                   model_weights_dir=model_weights_dir,
                                   model_weight_prefix=eval_prefix,
                                   dataset=ds_val[sensor],
                                   n_epochs=EVAL_N_EPOCHS,
                                   first_epoch=eval_first_step,
                                   steps_per_epoch=EVAL_STEPS_PER_EPOCH,
                                   k_epoch=25,
                                   csv_dir=LOGS_EXP_DIR + '/csv/' + val_or_test + '-' + sensor, 
                                   per_image=EVAL_PER_IMAGE, 
                                   verbose=0)

### 8.4 Comparison plots